In [1]:
import requests
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import itertools

In [2]:
query="2022-04-26T20:00:00"

## Define functions

Write the functions to pull data from the APIs

In [3]:
def get_weather_data(query:str):
    data_sets = ["air-temperature","rainfall","relative-humidity"]
    results={}
    for measure in data_sets:
        URL = "https://api.data.gov.sg/v1/environment/"+measure
        params={'date_time':query}
        r=requests.get(URL,params=params)
        results[measure]=r.json()
    return results

def get_taxi_data(query:str):
    URL = "https://api.data.gov.sg/v1/transport/taxi-availability"
    params={'date_time':query}
    r=requests.get(URL,params=params)
    return r.json()

### Pull data

TODO:



Assign each coordinate to nearest Euclidean distance weather station

Count availability for each weather station

In [63]:
weather_data=get_weather_data(query=query)
taxi_data=get_taxi_data(query=query)

In [64]:
taxi_coordinates = taxi_data['features'][0]['geometry']['coordinates']
station_dict = weather_data['air-temperature']['metadata']['stations']
taxi_array = np.array([t[::-1] for t in taxi_coordinates]) # taxi_coordinates are reversed, need to fix
station_array = np.array([[station_dict[i]['location']['latitude'],station_dict[i]['location']['longitude']] for i in range(len(station_dict))])

In [65]:
cartesian_prod = itertools.product(taxi_array,station_array)
diff = np.array([i[0]-i[1] for i in cartesian_prod])
distance = np.linalg.norm(diff,axis=1)

In [66]:
distance_matrix = distance.reshape((len(taxi_array),len(station_array),-1))
distance_matrix

array([[[0.24285869],
        [0.16062869],
        [0.34108052],
        ...,
        [0.132077  ],
        [0.22426301],
        [0.17926679]],

       [[0.23698088],
        [0.15510579],
        [0.33647987],
        ...,
        [0.12829506],
        [0.21748073],
        [0.17245633]],

       [[0.19289775],
        [0.11343203],
        [0.29875572],
        ...,
        [0.09934566],
        [0.1714949 ],
        [0.12677939]],

       ...,

       [[0.14116767],
        [0.21439751],
        [0.05436775],
        ...,
        [0.24861088],
        [0.22040777],
        [0.24747844]],

       [[0.14126218],
        [0.219976  ],
        [0.0793225 ],
        ...,
        [0.25907806],
        [0.21212831],
        [0.24321349]],

       [[0.15308234],
        [0.23014535],
        [0.07746946],
        ...,
        [0.26725287],
        [0.22606998],
        [0.25640292]]])

TODO: get nearest weather station for each taxi

In [ ]:
weather_data['air-temperature']

In [ ]:
print(weather_data['air-temperature']['metadata'])
print(weather_data['air-temperature']['items'])
print(weather_data['air-temperature']['api_info'])

In [ ]:
weather_data['rainfall']

In [ ]:
weather_data['relative-humidity']

# Modeling

Try different time lag for forecasting availability. 1 min, 15 min, 30 min etc.

# Serving Prediction

TODO: Decide on what we are trying to predict